In [73]:
import re
import pandas as pd
from itertools import combinations
from nltk.tokenize import sent_tokenize, word_tokenize

In [15]:
df = pd.read_json('./qangaroo_v1.1/medhop/train.json', orient='records')

In [16]:
df

,answer,candidates,id,query,supports
0,DB00072,"[DB00072, DB00294, DB00338, DB00341, DB00588, ...",MH_train_0,interacts_with DB00773?,[Induction of apoptosis of Beta cells of the p...
1,DB06822,"[DB00294, DB00313, DB00588, DB00755, DB00783, ...",MH_train_1,interacts_with DB09079?,[Prolonged treatment with bicalutamide induces...
2,DB00341,"[DB00035, DB00293, DB00322, DB00341, DB00351, ...",MH_train_2,interacts_with DB00083?,[Anti-Parkinson 's disease drugs and pharmacog...
3,DB01171,"[DB00619, DB00630, DB00783, DB00819, DB00989, ...",MH_train_3,interacts_with DB00083?,[A protein chip membrane-capture assay for bot...
4,DB01050,"[DB00035, DB00313, DB00338, DB00501, DB00820, ...",MH_train_4,interacts_with DB06813?,[P21554 cannabinoid receptor deficiency promot...
5,DB01200,"[DB00035, DB00290, DB00293, DB00338, DB00341, ...",MH_train_5,interacts_with DB06288?,[Role of monoamine oxidases in the exaggerated...
6,DB00203,"[DB00009, DB00035, DB00203, DB00588, DB00755, ...",MH_train_6,interacts_with DB00862?,[Synthetic delivery system for tuberculosis va...
7,DB01656,"[DB00290, DB00452, DB00630, DB00783, DB00977, ...",MH_train_7,interacts_with DB00773?,[Current researches on breast cancer epidemiol...
8,DB04844,"[DB00015, DB00341, DB00741, DB00820, DB01576, ...",MH_train_8,interacts_with DB01233?,[Chronic daily tadalafil prevents the corporal...
9,DB01182,"[DB00338, DB00351, DB00459, DB00588, DB00820, ...",MH_train_9,interacts_with DB00277?,[Nearly Complete Response of Brain Metastases ...


In [23]:
test_doc = df['supports'][0][0]

### Try to obtain a sentences and the args for one document

In [24]:
test_doc

'Induction of apoptosis of Beta cells of the pancreas by advanced glycation end-products , important mediators of chronic complications of diabetes mellitus . We herein report cytotoxicity of advanced glycation end-products ( AGEs ) on pancreatic beta cells . AGEs stimulated reactive oxygen species ( ROS ) generation but did not arrest proliferation of the P01308 -1 cell line . Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606 ( RAGE ) , and its expression increased after P51606 treatment . TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) . Reaction of P01308 -1 cells to Ki67 , which is a cellular marker for proliferation , was also increased after P51606 treatment . The ability of primary cultured islets to secrete ins

In [28]:
biomed_entity = re.compile("[ABCDEFGHIKMOPQS][0-9,A-Z]+[0-9]+")

In [38]:
sent_tokenize(test_doc)

['Induction of apoptosis of Beta cells of the pancreas by advanced glycation end-products , important mediators of chronic complications of diabetes mellitus .',
 'We herein report cytotoxicity of advanced glycation end-products ( AGEs ) on pancreatic beta cells .',
 'AGEs stimulated reactive oxygen species ( ROS ) generation but did not arrest proliferation of the P01308 -1 cell line .',
 'Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606 ( RAGE ) , and its expression increased after P51606 treatment .',
 'TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) .',
 'Reaction of P01308 -1 cells to Ki67 , which is a cellular marker for proliferation , was also increased after P51606 treatment .',
 'The ability of primary cultu

In [49]:
list(filter(lambda x: len(x[1]) > 0, 
       [(s, [(i, w) for i, w in enumerate(word_tokenize(s)) if biomed_entity.match(w) is not None]) 
        for s in sent_tokenize(test_doc)]))

[('AGEs stimulated reactive oxygen species ( ROS ) generation but did not arrest proliferation of the P01308 -1 cell line .',
  [(16, 'P01308')]),
 ('Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606 ( RAGE ) , and its expression increased after P51606 treatment .',
  [(12, 'P51606'), (22, 'P51606')]),
 ('TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) .',
  [(14, 'P01308'), (29, 'AGE2'), (34, 'AGE3'), (44, 'AGE1')]),
 ('Reaction of P01308 -1 cells to Ki67 , which is a cellular marker for proliferation , was also increased after P51606 treatment .',
  [(2, 'P01308'), (20, 'P51606')]),
 ('The ability of primary cultured islets to secrete insulin was retained even after P51606 treatment under either low or high glucose c

In [97]:
def extract_entities(sentence):
    unique_entities = set()
    entities = []
    
    for i, t in enumerate(word_tokenize(sentence)):
        if biomed_entity.match(t) is not None and t not in unique_entities:
            entities.append((i, t))
            unique_entities.add(set)
    return entities

def extract_medhop_instances(documents):
    instances = []
    for d in documents:
        documents_instances = list(filter(lambda x: len(x[1]) > 1, 
                                          [(s, extract_entities(s)) for s in sent_tokenize(test_doc)]))
        instances += documents_instances
    return instances       

In [98]:
df['medhop_instances'] = df['supports'].apply(extract_medhop_instances)

In [99]:
df['medhop_instances'][0]

[('Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606 ( RAGE ) , and its expression increased after P51606 treatment .',
  [(12, 'P51606'), (22, 'P51606')]),
 ('TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) .',
  [(14, 'P01308'), (29, 'AGE2'), (34, 'AGE3'), (44, 'AGE1')]),
 ('Reaction of P01308 -1 cells to Ki67 , which is a cellular marker for proliferation , was also increased after P51606 treatment .',
  [(2, 'P01308'), (20, 'P51606')]),
 ('Moreover , antibody array showed that Q06609 and P43351 were significantly decreased in AGE2-treated P01308 -1 cells .',
  [(6, 'Q06609'), (8, 'P43351'), (13, 'AGE2-treated'), (14, 'P01308')]),
 ('Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606

### Get tuples

In [101]:
df['medhop_instances'][0][1][1]

[(14, 'P01308'), (29, 'AGE2'), (34, 'AGE3'), (44, 'AGE1')]

In [102]:
list(combinations(df['medhop_instances'][0][2][1], 2))

[((2, 'P01308'), (20, 'P51606'))]

In [104]:
def get_binary_medhop_instances(medhop_instances):
    binary_instances = []
    for instance in medhop_instances:
        pairs = list(combinations(instance[1], 2))
        binary_instances += [(instance[0], pair) for pair in pairs]
    return binary_instances

In [105]:
df['binary_medhop_instances'] = df['medhop_instances'].apply(get_binary_medhop_instances)

In [110]:
df['binary_medhop_instances'][2]

[('Pancreatic beta cell lines or primary cultured islets possess a receptor for P51606 ( RAGE ) , and its expression increased after P51606 treatment .',
  ((12, 'P51606'), (22, 'P51606'))),
 ('TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) .',
  ((14, 'P01308'), (29, 'AGE2'))),
 ('TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incubated with BSA conjugated with glyceraldehyde ( AGE2 ) or glucoaldehyde ( AGE3 ) , compared with those conjugated with glucose ( AGE1 ) .',
  ((14, 'P01308'), (34, 'AGE3'))),
 ('TUNEL staining and FACS analysis using annexin V/PI antibodies showed that apoptosis increased in P01308 -1 cells or primary cultured islets when incu